In [18]:
import pandas as pd
import urllib
import io

In [16]:
# Download and import data

download = True # If download is true download and save data, else just read data

if(download):
    data_url = "https://www.dropbox.com/s/mxlqmphe9dxtw8y/data_v3.csv?dl=1"
    df = pd.read_csv(data_url)
    df.to_csv("data.csv", index = False)
else:
    df = pd.read_csv("data.csv")

KeyboardInterrupt: 

In [19]:
url = "http://export.arxiv.org/oai2?verb=ListSets"
u = urllib.request.urlopen(url, data = None)
f = io.TextIOWrapper(u,encoding='utf-8')
text = f.read()
soup = BeautifulSoup(text, 'xml')
all_cat = [sp.text for sp in soup.findAll("setSpec")]

f = open("all_cat_v01.txt", "w")
f.write(",".join(all_cat))
f.close()

NameError: name 'BeautifulSoup' is not defined

In [14]:
df.shape

(2098183, 6)

In [11]:
df.drop_duplicates().shape

(2098183, 6)

In [12]:
df.dropna().shape

(2098180, 6)